In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms as T
import cv2
from tqdm.notebook import tqdm
import pandas as pd

from self_sup_data.mvtec import SelfSupMVTecDataset, CLASS_NAMES, OBJECTS, TEXTURES
from model.resnet import resnet18_enc_dec
from experiments.training_utils import train_and_save_model
from train_mvtec import SETTINGS
from experiments.mvtec_tasks import test_real_anomalies


def test(class_name, test_dat, setting, device, preact=False, pool=True, final=True, show=False):
    if final:
        fname = os.path.join(model_dir, setting.get('out_dir'), class_name, 'final_' + class_name + '_' + setting.get('fname'))
    else: 
        fname = os.path.join(model_dir, setting.get('out_dir'), class_name, class_name + '_' + setting.get('fname'))
    print(fname)
    if not os.path.exists(fname):
        return np.nan, np.nan, np.nan, np.nan, np.nan

    model = resnet18_enc_dec(num_classes=1, preact=preact, pool=pool,
                             final_activation=setting.get('final_activation')).to(device)
    if final:
        model.load_state_dict(torch.load(fname))
    else:
        model.load_state_dict(torch.load(fname).get('model_state_dict'))
    
    sample_ap, sample_auroc, pixel_ap, pixel_auroc, pixel_pro = test_real_anomalies(model, test_dat, 
                        device=device, batch_size=16, show=show, full_size = class_name in OBJECTS)
    
    return sample_ap, sample_auroc, pixel_ap, pixel_auroc, pixel_pro


model_dir = 'put/your/path/here'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
out_dir = 'put/your/path/here'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

data_dir = 'put/your/path/here'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

Using cuda


# Run evaluation

In [ ]:
modes = list(SETTINGS.keys())

sample_ap_df = pd.DataFrame(columns=['class', 'category',] + modes)
sample_auroc_df = pd.DataFrame(columns=['class', 'category',] + modes)
pixel_ap_df = pd.DataFrame(columns=['class', 'category',] + modes)
pixel_auroc_df = pd.DataFrame(columns=['class', 'category',] + modes)
pixel_pro_df = pd.DataFrame(columns=['class', 'category',] + modes)

for class_name in tqdm(CLASS_NAMES):
    test_dat = SelfSupMVTecDataset(root_path=data_dir, class_name=class_name, is_train=False, 
                                   low_res=256, download=False)
    kind = 'texture' if class_name in TEXTURES else 'object'
    sample_aps, sample_aurocs = {'category': class_name, 'class': kind}, {'category': class_name,'class': kind}
    sample_aps_new, sample_aurocs_new = {'category': class_name, 'class': kind}, {'category': class_name,'class': kind}
    pixel_aps, pixel_aurocs = {'category': class_name,'class': kind}, {'category': class_name,'class': kind}
    pixel_pros = {'category': class_name,'class': kind}
    for mode in modes:
        sample_aps[mode], sample_aurocs[mode], pixel_aps[mode], pixel_aurocs[mode], pixel_pros[mode] = test(
                class_name, test_dat, SETTINGS.get(mode), device, preact=False, pool=True, final=True, show=True)
    del test_dat
    gc.collect()
        
    sample_ap_df = sample_ap_df.append(sample_aps, ignore_index=True)
    sample_auroc_df = sample_auroc_df.append(sample_aurocs, ignore_index=True)
    pixel_ap_df = pixel_ap_df.append(pixel_aps, ignore_index=True)
    pixel_auroc_df = pixel_auroc_df.append(pixel_aurocs, ignore_index=True)
    pixel_pro_df = pixel_pro_df.append(pixel_pros, ignore_index=True)

In [6]:
sample_auroc_df.to_csv(os.path.join(out_dir, 'sample_auroc.csv'), index=False)
pixel_auroc_df.to_csv(os.path.join(out_dir, 'pixel_auroc.csv'), index=False)
pixel_pro_df.to_csv(os.path.join(out_dir, 'pixel_pro.csv'), index=False)

# Aggregate results

Produces tables for the paper by combining results from our experiments and SOTA.

## Image-level AUROC

In [10]:
# copied from paper (just used for automatic latex table making below)
cutpaste_texture = pd.DataFrame().append(
    {'carpet': r'93.1 {\tiny $\pm$ 1.1}', 'grid':r'99.9 {\tiny $\pm$ 0.1}', 
     'leather':r'100.0 {\tiny $\pm$ 0.0}', 'tile':r'93.4 {\tiny $\pm$ 1.0}', 
     'wood':r'98.6 {\tiny $\pm$ 0.5}'}, 
    ignore_index=True).transpose().reset_index()
cutpaste_texture.columns = ['category','CutPaste (3-way) [TODO cite]']
cutpaste_texture['class'] = 'texture'
cutpaste_object = pd.DataFrame().append( 
    {'bottle': r'98.3 {\tiny $\pm$ 0.5}', 'cable':r'80.6 {\tiny $\pm$ 0.5}', 
     'capsule':r'96.2 {\tiny $\pm$ 0.5}', 'hazelnut':r'97.3 {\tiny $\pm$ 0.3}', 
     'metal_nut':r'99.3 {\tiny $\pm$ 0.2}', 'pill':r'92.4 {\tiny $\pm$ 1.3}', 
     'screw':r'86.3 {\tiny $\pm$ 1.0}', 'toothbrush':r'98.3 {\tiny $\pm$ 0.9}',
     'transistor': r'95.5 {\tiny $\pm$ 0.5}', 'zipper': r'99.4 {\tiny $\pm$ 0.2}'}, ignore_index=True).transpose().reset_index()
cutpaste_object.columns = ['category','CutPaste (3-way) [TODO cite]']
cutpaste_object['class'] = 'object'
cutpaste = cutpaste_object.append(cutpaste_texture)
cutpaste = cutpaste.append({'category':'zz average', 'class':'object', 
                            'CutPaste (3-way) [TODO cite]':r'94.3 {\tiny $\pm$ 0.6}'}, 
                           ignore_index=True)
cutpaste = cutpaste.append({'category':'zz average', 'class':'texture', 
                            'CutPaste (3-way) [TODO cite]':r'97.0 {\tiny $\pm$ 0.5}'}, 
                           ignore_index=True)
cutpaste = cutpaste.append({'category':'zz average', 'class':'total', 
                            'CutPaste (3-way) [TODO cite]':r'95.2 {\tiny $\pm$ 0.6}'}, 
                           ignore_index=True)
cutpaste

,category,CutPaste (3-way) [TODO cite],class
0,bottle,98.3 {\tiny $\pm$ 0.5},object
1,cable,80.6 {\tiny $\pm$ 0.5},object
2,capsule,96.2 {\tiny $\pm$ 0.5},object
3,hazelnut,97.3 {\tiny $\pm$ 0.3},object
4,metal_nut,99.3 {\tiny $\pm$ 0.2},object
5,pill,92.4 {\tiny $\pm$ 1.3},object
6,screw,86.3 {\tiny $\pm$ 1.0},object
7,toothbrush,98.3 {\tiny $\pm$ 0.9},object
8,transistor,95.5 {\tiny $\pm$ 0.5},object
9,zipper,99.4 {\tiny $\pm$ 0.2},object


In [11]:
pd.options.display.float_format = '{:3.1f}'.format

sample_table = sample_auroc_df.reset_index().rename(columns={'CutPaste':'CutPaste (end-to-end)'})
# we use mixed for textures and normal for objects
shift_cols = [col for col in sample_table.columns if 'Shift' in col and not '-M' in col]
shift_m_cols = [col for col in sample_table.columns if 'Shift' in col and '-M' in col]
sample_table = (sample_table[sample_table['class'] == 'object'].drop(columns=shift_m_cols)
               .append(sample_table[sample_table['class']  == 'texture'].drop(columns=shift_cols)
                       .rename(columns=lambda x: x.replace('-M', ''))))

sample_table = 100 * (sample_table.set_index(['class', 'category']))

sample_class_mean = sample_table.groupby(['class']).mean() 
sample_class_mean['category'] = 'zz average' # so it appears last 
sample_mean = pd.DataFrame(sample_table.mean(axis=0)).transpose()
sample_mean['class'] = 'total'
sample_mean['category'] = 'zz average'
sample_table = sample_table.append(sample_class_mean.reset_index().set_index(['class', 'category']))
sample_table = sample_table.append(sample_mean.reset_index().set_index(['class', 'category'])).reset_index()

cols_shift = sample_table.loc[: , 'Shift':'Shift-123425']
cols_shift_int = sample_table.loc[: , 'Shift-Intensity':'Shift-Intensity-123425']
cols_shift_raw_int = sample_table.loc[: , 'Shift-Raw-Intensity':'Shift-Raw-Intensity-123425']

merge_func = lambda x: r'{:3.1f} {{\tiny $\pm$ {:3.1f}}}'.format(*x)
sample_table['Shift'] = list(map(merge_func, zip(cols_shift.mean(axis=1),  cols_shift.std(axis=1))))
sample_table['Shift-Intensity'] = list(map(merge_func, zip(cols_shift_int.mean(axis=1),  cols_shift_int.std(axis=1))))
sample_table['Shift-Raw-Intensity'] = list(map(merge_func, zip(cols_shift_raw_int.mean(axis=1),  cols_shift_raw_int.std(axis=1))))

sample_table = sample_table[['class', 'category',
                             'CutPaste (end-to-end)', 'FPI', 'FPI-Poisson', 
                             'Shift', 'Shift-Raw-Intensity', 'Shift-Intensity']].rename(
    columns={'Shift':'Ours (binary)', 'Shift-Intensity':'Ours (logistic continuous)', 'Shift-Raw-Intensity':'Ours (continous)'})
# first is just getting rid of nans caused by cutpaste table missing other columns
sample_table = cutpaste.append(sample_table.reset_index()).groupby(['class', 'category']).first()
sample_table = sample_table.sort_values(['class', 'category']).drop(columns='index')

print(sample_table.to_latex(escape=False))

\begin{tabular}{lllrrrlll}
\toprule
      &            & CutPaste (3-way) [TODO cite] &  CutPaste (end-to-end) &  FPI &  FPI-Poisson &            Ours (binary) &         Ours (continous) & Ours (logistic continuous) \\
class & category &                              &                        &      &              &                          &                          &                            \\
\midrule
object & bottle &       98.3 {\tiny $\pm$ 0.5} &                  100.0 & 90.2 &         97.6 &   97.6 {\tiny $\pm$ 0.2} &   97.5 {\tiny $\pm$ 0.2} &     97.7 {\tiny $\pm$ 0.3} \\
      & cable &       80.6 {\tiny $\pm$ 0.5} &                   75.4 & 68.0 &         68.9 &   92.1 {\tiny $\pm$ 2.4} &   90.2 {\tiny $\pm$ 3.0} &     94.5 {\tiny $\pm$ 1.0} \\
      & capsule &       96.2 {\tiny $\pm$ 0.5} &                   89.2 & 87.5 &         84.9 &   93.2 {\tiny $\pm$ 0.8} &   92.8 {\tiny $\pm$ 2.2} &     95.2 {\tiny $\pm$ 1.7} \\
      & hazelnut &       97.3 {\tiny $\pm$ 0.3} &    

In [12]:
sample_table

CutPaste (3-way) [TODO cite]  CutPaste (end-to-end)  FPI  \
class   category                                                              
object  bottle           98.3 {\tiny $\pm$ 0.5}                  100.0 90.2   
        cable            80.6 {\tiny $\pm$ 0.5}                   75.4 68.0   
        capsule          96.2 {\tiny $\pm$ 0.5}                   89.2 87.5   
        hazelnut         97.3 {\tiny $\pm$ 0.3}                   81.4 86.0   
        metal_nut        99.3 {\tiny $\pm$ 0.2}                   70.6 88.4   
        pill             92.4 {\tiny $\pm$ 1.3}                   90.3 71.8   
        screw            86.3 {\tiny $\pm$ 1.0}                   65.5 61.2   
        toothbrush       98.3 {\tiny $\pm$ 0.9}                   96.7 85.8   
        transistor       95.5 {\tiny $\pm$ 0.5}                   88.2 79.6   
        zipper           99.4 {\tiny $\pm$ 0.2}                   98.7 97.7   
        zz average       94.3 {\tiny $\pm$ 0.6}                   85.6 81.6   
texture carpet           93.1 {\tiny $\pm$ 1.1}                   53.1 56.0   
        grid             99.9 {\tiny $\pm$ 0.1}                   99.7 99.5   
        leather         100.0 {\tiny $\pm$ 0.0}                   86.6 91.7   
        tile             93.4 {\tiny $\pm$ 1.0}                   87.8 90.2   
        wood             98.6 {\tiny $\pm$ 0.5}                   84.6 74.4   
        zz average       97.0 {\tiny $\pm$ 0.5}                   82.4 82.4   
total   zz average       95.2 {\tiny $\pm$ 0.6}                   84.5 81.9   

                    FPI-Poisson            Ours (binary)  \
class   category                                           
object  bottle             97.6   97.6 {\tiny $\pm$ 0.2}   
        cable              68.9   92.1 {\tiny $\pm$ 2.4}   
        capsule            84.9   93.2 {\tiny $\pm$ 0.8}   
        hazelnut           82.7   93.5 {\tiny $\pm$ 1.9}   
        metal_nut          98.9   99.4 {\tiny $\pm$ 0.3}   
        pill               86.3   97.0 {\tiny $\pm$ 0.9}   
        screw              74.7   90.3 {\tiny $\pm$ 1.2}   
        toothbrush         93.1  100.0 {\tiny $\pm$ 0.0}   
        transistor         90.1   93.5 {\tiny $\pm$ 0.9}   
        zipper             99.8   99.8 {\tiny $\pm$ 0.1}   
        zz average         87.7   95.6 {\tiny $\pm$ 0.5}   
texture carpet             65.6   85.6 {\tiny $\pm$ 7.6}   
        grid              100.0   99.9 {\tiny $\pm$ 0.1}   
        leather           100.0   99.9 {\tiny $\pm$ 0.1}   
        tile               98.4   99.7 {\tiny $\pm$ 0.2}   
        wood               91.9   96.7 {\tiny $\pm$ 1.2}   
        zz average         91.2   96.4 {\tiny $\pm$ 1.4}   
total   zz average         88.9   95.9 {\tiny $\pm$ 0.7}   

                           Ours (continous) Ours (logistic continuous)  
class   category                                                        
object  bottle       97.5 {\tiny $\pm$ 0.2}     97.7 {\tiny $\pm$ 0.3}  
        cable        90.2 {\tiny $\pm$ 3.0}     94.5 {\tiny $\pm$ 1.0}  
        capsule      92.8 {\tiny $\pm$ 2.2}     95.2 {\tiny $\pm$ 1.7}  
        hazelnut     89.3 {\tiny $\pm$ 4.9}     94.7 {\tiny $\pm$ 1.1}  
        metal_nut    94.6 {\tiny $\pm$ 2.1}     98.7 {\tiny $\pm$ 0.7}  
        pill         94.3 {\tiny $\pm$ 1.1}     99.2 {\tiny $\pm$ 0.6}  
        screw        90.1 {\tiny $\pm$ 0.9}     90.2 {\tiny $\pm$ 1.4}  
        toothbrush   99.6 {\tiny $\pm$ 0.5}    100.0 {\tiny $\pm$ 0.0}  
        transistor   92.8 {\tiny $\pm$ 2.2}     95.1 {\tiny $\pm$ 0.2}  
        zipper       99.5 {\tiny $\pm$ 0.7}     99.8 {\tiny $\pm$ 0.1}  
        zz average   94.1 {\tiny $\pm$ 1.0}     96.5 {\tiny $\pm$ 0.4}  
texture carpet       90.9 {\tiny $\pm$ 2.2}     95.6 {\tiny $\pm$ 0.6}  
        grid         98.5 {\tiny $\pm$ 3.3}     99.9 {\tiny $\pm$ 0.1}  
        leather     100.0 {\tiny $\pm$ 0.0}     99.9 {\tiny $\pm$ 0.1}  
        tile        100.0 {\tiny $\pm$ 0.0}    100.0 {\tiny $\pm$ 0.0

## Pixel-level AUROC

In [13]:
# copied from paper (just used for automatic latex table making below)
cutpaste_texture = pd.DataFrame().append(
    {'carpet': r'98.3 {\tiny $\pm$ 0.0}', 'grid':r'97.5 {\tiny $\pm$ 0.1}', 
     'leather':r'99.5 {\tiny $\pm$ 0.0}', 'tile':r'90.5 {\tiny $\pm$ 0.2}', 
     'wood':r'95.5 {\tiny $\pm$ 0.1}'}, 
    ignore_index=True).transpose().reset_index()
cutpaste_texture.columns = ['category','CutPaste (3-way) [TODO cite and add std]']
cutpaste_texture['class'] = 'texture'
cutpaste_object = pd.DataFrame().append( 
    {'bottle': r'97.6 {\tiny $\pm$ 0.1}', 'cable':r'90.0 {\tiny $\pm$ 0.2}', 
     'capsule':r'97.4 {\tiny $\pm$ 0.1}', 'hazelnut':r'97.3 {\tiny $\pm$ 0.1}', 
     'metal_nut':r'93.1 {\tiny $\pm$ 0.4}', 'pill':r'95.7 {\tiny $\pm$ 0.1}', 
     'screw':r'96.7 {\tiny $\pm$ 0.1}', 'toothbrush':r'98.1 {\tiny $\pm$ 0.0}',
     'transistor': r'93.0 {\tiny $\pm$ 0.2}', 'zipper': r'99.3 {\tiny $\pm$ 0.0}'}, ignore_index=True).transpose().reset_index()
cutpaste_object.columns = ['category','CutPaste (3-way) [TODO cite and add std]']
cutpaste_object['class'] = 'object'
cutpaste = cutpaste_object.append(cutpaste_texture)
cutpaste = cutpaste.append({'category':'zz average', 'class':'object', 
                            'CutPaste (3-way) [TODO cite and add std]':r'95.8 {\tiny $\pm$ 0.1}'}, 
                           ignore_index=True)
cutpaste = cutpaste.append({'category':'zz average', 'class':'texture', 
                            'CutPaste (3-way) [TODO cite and add std]':r'96.3 {\tiny $\pm$ 0.1}'}, 
                           ignore_index=True)
cutpaste = cutpaste.append({'category':'zz average', 'class':'total', 
                            'CutPaste (3-way) [TODO cite and add std]':r'96.0 {\tiny $\pm$ 0.1}'}, 
                           ignore_index=True)
cutpaste

,category,CutPaste (3-way) [TODO cite and add std],class
0,bottle,97.6 {\tiny $\pm$ 0.1},object
1,cable,90.0 {\tiny $\pm$ 0.2},object
2,capsule,97.4 {\tiny $\pm$ 0.1},object
3,hazelnut,97.3 {\tiny $\pm$ 0.1},object
4,metal_nut,93.1 {\tiny $\pm$ 0.4},object
5,pill,95.7 {\tiny $\pm$ 0.1},object
6,screw,96.7 {\tiny $\pm$ 0.1},object
7,toothbrush,98.1 {\tiny $\pm$ 0.0},object
8,transistor,93.0 {\tiny $\pm$ 0.2},object
9,zipper,99.3 {\tiny $\pm$ 0.0},object


In [14]:
# copied from paper (just used for automatic latex table making below)
padim_texture = pd.DataFrame().append(
    {'carpet': 99.1, 'grid':97.3, 'leather':99.2, 'tile':94.1, 'wood':94.9}, 
    ignore_index=True).transpose().reset_index()
padim_texture.columns = ['category','PaDiM-WR50-Rd550 [TODO cite]']
padim_texture['class'] = 'texture'
padim_object = pd.DataFrame().append( 
    {'bottle': 98.3, 'cable':96.7, 'capsule':98.5, 'hazelnut':98.2, 
     'metal_nut':97.2, 'pill':95.7, 'screw':98.5, 'toothbrush':98.8,
     'transistor': 97.5, 'zipper': 98.5}, ignore_index=True).transpose().reset_index()
padim_object.columns = ['category','PaDiM-WR50-Rd550 [TODO cite]']
padim_object['class'] = 'object'
padim = padim_object.append(padim_texture)
padim = padim.append({'category':'zz average', 'class':'object', 
                           'PaDiM-WR50-Rd550 [TODO cite]':97.8}, 
                           ignore_index=True)
padim = padim.append({'category':'zz average', 'class':'texture', 
                            'PaDiM-WR50-Rd550 [TODO cite]':96.9}, 
                           ignore_index=True)
padim = padim.append({'category':'zz average', 'class':'total', 
                            'PaDiM-WR50-Rd550 [TODO cite]':97.5}, 
                           ignore_index=True)
padim

,category,PaDiM-WR50-Rd550 [TODO cite],class
0,bottle,98.3,object
1,cable,96.7,object
2,capsule,98.5,object
3,hazelnut,98.2,object
4,metal_nut,97.2,object
5,pill,95.7,object
6,screw,98.5,object
7,toothbrush,98.8,object
8,transistor,97.5,object
9,zipper,98.5,object


In [15]:
pd.options.display.float_format = '{:3.1f}'.format

pixel_table = pixel_auroc_df.reset_index().rename(columns={'CutPaste':'CutPaste (end-to-end)'})
# we use mixed for textures and normal for objects
shift_cols = [col for col in pixel_table.columns if 'Shift' in col and not '-M' in col]
shift_m_cols = [col for col in pixel_table.columns if 'Shift' in col and '-M' in col]
pixel_table = (pixel_table[pixel_table['class'] == 'object'].drop(columns=shift_m_cols)
               .append(pixel_table[pixel_table['class']  == 'texture'].drop(columns=shift_cols)
                       .rename(columns=lambda x: x.replace('-M', ''))))

pixel_table = 100 * (pixel_table.set_index(['class', 'category']))

pixel_class_mean = pixel_table.groupby(['class']).mean() 
pixel_class_mean['category'] = 'zz average' # so it appears last 
pixel_mean = pd.DataFrame(pixel_table.mean(axis=0)).transpose()
pixel_mean['class'] = 'total'
pixel_mean['category'] = 'zz average'
pixel_table = pixel_table.append(pixel_class_mean.reset_index().set_index(['class', 'category']))
pixel_table = pixel_table.append(pixel_mean.reset_index().set_index(['class', 'category'])).reset_index()

cols_shift = pixel_table.loc[: , 'Shift':'Shift-123425']
cols_shift_int = pixel_table.loc[: , 'Shift-Intensity':'Shift-Intensity-123425']
cols_shift_raw_int = pixel_table.loc[: , 'Shift-Raw-Intensity':'Shift-Raw-Intensity-123425']

merge_func = lambda x: r'{:3.1f} {{\tiny $\pm$ {:3.1f}}}'.format(*x)
pixel_table['Shift'] = list(map(merge_func, zip(cols_shift.mean(axis=1),  cols_shift.std(axis=1))))
pixel_table['Shift-Intensity'] = list(map(merge_func, zip(cols_shift_int.mean(axis=1),  cols_shift_int.std(axis=1))))
pixel_table['Shift-Raw-Intensity'] = list(map(merge_func, zip(cols_shift_raw_int.mean(axis=1),  cols_shift_raw_int.std(axis=1))))

pixel_table = pixel_table[['class', 'category',
                             'CutPaste (end-to-end)', 'FPI', 'FPI-Poisson', 
                             'Shift','Shift-Raw-Intensity', 'Shift-Intensity']].rename(
    columns={'Shift':'Ours (binary)', 'Shift-Intensity':'Ours (logistic continuous)', 'Shift-Raw-Intensity':'Ours (continous)'})
# first is just getting rid of nans caused by cutpaste table missing other columns
pixel_table = padim.append(cutpaste).append(pixel_table.reset_index()).groupby(['class', 'category']).first()
pixel_table = pixel_table.sort_values(['class', 'category']).drop(columns='index')

print(pixel_table.to_latex(escape=False))

\begin{tabular}{llrlrrrlll}
\toprule
      &            &  PaDiM-WR50-Rd550 [TODO cite] & CutPaste (3-way) [TODO cite and add std] &  CutPaste (end-to-end) &  FPI &  FPI-Poisson &           Ours (binary) &        Ours (continous) & Ours (logistic continuous) \\
class & category &                               &                                          &                        &      &              &                         &                         &                            \\
\midrule
object & bottle &                          98.3 &                   97.6 {\tiny $\pm$ 0.1} &                   97.7 & 91.8 &         93.1 &  98.4 {\tiny $\pm$ 0.2} &  97.3 {\tiny $\pm$ 0.5} &     98.3 {\tiny $\pm$ 0.1} \\
      & cable &                          96.7 &                   90.0 {\tiny $\pm$ 0.2} &                   81.0 & 66.5 &         70.2 &  93.3 {\tiny $\pm$ 3.4} &  91.0 {\tiny $\pm$ 2.8} &     96.0 {\tiny $\pm$ 1.4} \\
      & capsule &                          98.5 &               

In [16]:
pixel_table

PaDiM-WR50-Rd550 [TODO cite]  \
class   category                                   
object  bottle                              98.3   
        cable                               96.7   
        capsule                             98.5   
        hazelnut                            98.2   
        metal_nut                           97.2   
        pill                                95.7   
        screw                               98.5   
        toothbrush                          98.8   
        transistor                          97.5   
        zipper                              98.5   
        zz average                          97.8   
texture carpet                              99.1   
        grid                                97.3   
        leather                             99.2   
        tile                                94.1   
        wood                                94.9   
        zz average                          96.9   
total   zz average                          97.5   

                   CutPaste (3-way) [TODO cite and add std]  \
class   category                                              
object  bottle                       97.6 {\tiny $\pm$ 0.1}   
        cable                        90.0 {\tiny $\pm$ 0.2}   
        capsule                      97.4 {\tiny $\pm$ 0.1}   
        hazelnut                     97.3 {\tiny $\pm$ 0.1}   
        metal_nut                    93.1 {\tiny $\pm$ 0.4}   
        pill                         95.7 {\tiny $\pm$ 0.1}   
        screw                        96.7 {\tiny $\pm$ 0.1}   
        toothbrush                   98.1 {\tiny $\pm$ 0.0}   
        transistor                   93.0 {\tiny $\pm$ 0.2}   
        zipper                       99.3 {\tiny $\pm$ 0.0}   
        zz average                   95.8 {\tiny $\pm$ 0.1}   
texture carpet                       98.3 {\tiny $\pm$ 0.0}   
        grid                         97.5 {\tiny $\pm$ 0.1}   
        leather                      99.5 {\tiny $\pm$ 0.0}   
        tile                         90.5 {\tiny $\pm$ 0.2}   
        wood                         95.5 {\tiny $\pm$ 0.1}   
        zz average                   96.3 {\tiny $\pm$ 0.1}   
total   zz average                   96.0 {\tiny $\pm$ 0.1}   

                    CutPaste (end-to-end)  FPI  FPI-Poisson  \
class   category                                              
object  bottle                       97.7 91.8         93.1   
        cable                        81.0 66.5         70.2   
        capsule                      97.5 95.9         90.2   
        hazelnut                     94.8 89.8         97.0   
        metal_nut                    68.1 96.2         95.4   
        pill                         98.1 62.3         95.3   
        screw                        90.7 90.4         92.8   
        toothbrush                   95.7 81.8         81.3   
        transistor                   85.9 78.5         86.9   
        zipper                       92.9 91.8         93.8   
        zz average                   90.2 84.5         89.6   
texture carpet                       83.3 70.8         97.2   
        grid                         97.6 94.2         98.9   
        leather                      96.4 88.3         99.2   
        tile                         72.7 65.0         98.0   
        wood                         84.0 71.1         91.1   
        zz average                   86.8 77.9         96.9   
total   zz average                   89.1 82.3         92.0   

                             Ours (binary)        Ours (continous)  \
class   category                                                     
object  bottle      98.4 {\tiny $\pm$ 0.2}  97.3 {\tiny $\pm$ 0.5}   
        cable       93.3 {\tiny $\pm$ 3.4}  91.0 {\tiny $\pm$ 2.8}   
        capsule     98.1 {\tiny $\pm$ 0.2}  91.6 {\tiny $\pm$ 5.6}   
        hazelnut    97.2 {\tiny $\pm$ 0.6}  97.7 {\tiny $\pm$ 0.6}   
        metal_nut   98.2 {\tiny $\pm$

## AU-PRO

In [17]:
# copied from paper (just used for automatic latex table making below)
padim_texture = pd.DataFrame().append(
    {'carpet': 96.2, 'grid':94.6, 'leather':97.8, 'tile':86.0, 'wood':91.1}, 
    ignore_index=True).transpose().reset_index()
padim_texture.columns = ['category','PaDiM-WR50-Rd550 [TODO cite]']
padim_texture['class'] = 'texture'
padim_object = pd.DataFrame().append( 
    {'bottle': 94.8, 'cable':88.8, 'capsule':93.5, 'hazelnut':92.6, 
     'metal_nut':85.6, 'pill':92.7, 'screw':94.4, 'toothbrush':93.1,
     'transistor': 84.5, 'zipper': 95.9}, ignore_index=True).transpose().reset_index()
padim_object.columns = ['category','PaDiM-WR50-Rd550 [TODO cite]']
padim_object['class'] = 'object'
padim = padim_object.append(padim_texture)
padim = padim.append({'category':'zz average', 'class':'object', 
                           'PaDiM-WR50-Rd550 [TODO cite]':91.6}, 
                           ignore_index=True)
padim = padim.append({'category':'zz average', 'class':'texture', 
                            'PaDiM-WR50-Rd550 [TODO cite]':93.2}, 
                           ignore_index=True)
padim = padim.append({'category':'zz average', 'class':'total', 
                            'PaDiM-WR50-Rd550 [TODO cite]':92.1}, 
                           ignore_index=True)
padim

,category,PaDiM-WR50-Rd550 [TODO cite],class
0,bottle,94.8,object
1,cable,88.8,object
2,capsule,93.5,object
3,hazelnut,92.6,object
4,metal_nut,85.6,object
5,pill,92.7,object
6,screw,94.4,object
7,toothbrush,93.1,object
8,transistor,84.5,object
9,zipper,95.9,object


In [20]:
pd.options.display.float_format = '{:3.1f}'.format

pixel_pro_table = pixel_pro_df.reset_index().rename(columns={'CutPaste':'CutPaste (end-to-end)'})
# we use mixed for textures and normal for objects
shift_cols = [col for col in pixel_pro_table.columns if 'Shift' in col and not '-M' in col]
shift_m_cols = [col for col in pixel_pro_table.columns if 'Shift' in col and '-M' in col]
pixel_pro_table = (pixel_pro_table[pixel_pro_table['class'] == 'object'].drop(columns=shift_m_cols)
               .append(pixel_pro_table[pixel_pro_table['class']  == 'texture'].drop(columns=shift_cols)
                       .rename(columns=lambda x: x.replace('-M', ''))))

pixel_pro_table = 100 * (pixel_pro_table.set_index(['class', 'category']))

pixel_class_mean = pixel_pro_table.groupby(['class']).mean() 
pixel_class_mean['category'] = 'zz average' # so it appears last 
pixel_mean = pd.DataFrame(pixel_pro_table.mean(axis=0)).transpose()
pixel_mean['class'] = 'total'
pixel_mean['category'] = 'zz average'
pixel_pro_table = pixel_pro_table.append(pixel_class_mean.reset_index().set_index(['class', 'category']))
pixel_pro_table = pixel_pro_table.append(pixel_mean.reset_index().set_index(['class', 'category'])).reset_index()

cols_shift = pixel_pro_table.loc[: , 'Shift':'Shift-123425']
cols_shift_int = pixel_pro_table.loc[: , 'Shift-Intensity':'Shift-Intensity-123425']
cols_shift_raw_int = pixel_pro_table.loc[: , 'Shift-Raw-Intensity':'Shift-Raw-Intensity-123425']

merge_func = lambda x: r'{:3.1f} {{\tiny $\pm$ {:3.1f}}}'.format(*x)
pixel_pro_table['Shift'] = list(map(merge_func, zip(cols_shift.mean(axis=1),  cols_shift.std(axis=1))))
pixel_pro_table['Shift-Intensity'] = list(map(merge_func, zip(cols_shift_int.mean(axis=1),  cols_shift_int.std(axis=1))))
pixel_pro_table['Shift-Raw-Intensity'] = list(map(merge_func, zip(cols_shift_raw_int.mean(axis=1),  cols_shift_raw_int.std(axis=1))))

pixel_pro_table = pixel_pro_table[['class', 'category',
                             'CutPaste (end-to-end)', 'FPI', 'FPI-Poisson', 
                             'Shift','Shift-Raw-Intensity', 'Shift-Intensity']].rename(
    columns={'Shift':'Ours (binary)', 'Shift-Intensity':'Ours (logistic continuous)', 'Shift-Raw-Intensity':'Ours (continous)'})
# first is just getting rid of nans caused by cutpaste table missing other columns
pixel_pro_table = padim.append(pixel_pro_table.reset_index()).groupby(['class', 'category']).first()
pixel_pro_table = pixel_pro_table.sort_values(['class', 'category']).drop(columns='index')

print(pixel_pro_table.to_latex(escape=False))

\begin{tabular}{llrrrrlll}
\toprule
      &            &  PaDiM-WR50-Rd550 [TODO cite] &  CutPaste (end-to-end) &  FPI &  FPI-Poisson &            Ours (binary) &         Ours (continous) & Ours (logistic continuous) \\
class & category &                               &                        &      &              &                          &                          &                            \\
\midrule
object & bottle &                          94.8 &                   91.2 & 66.0 &         79.0 &   93.0 {\tiny $\pm$ 0.9} &   89.9 {\tiny $\pm$ 1.1} &     92.9 {\tiny $\pm$ 0.3} \\
      & cable &                          88.8 &                   59.8 & 51.9 &         55.7 &   87.6 {\tiny $\pm$ 3.4} &   85.4 {\tiny $\pm$ 2.1} &     89.9 {\tiny $\pm$ 1.0} \\
      & capsule &                          93.5 &                   83.5 & 79.9 &         67.6 &   91.8 {\tiny $\pm$ 0.8} &   79.9 {\tiny $\pm$ 9.0} &     91.4 {\tiny $\pm$ 2.2} \\
      & hazelnut &                          92.6

In [21]:
pixel_pro_table

PaDiM-WR50-Rd550 [TODO cite]  CutPaste (end-to-end)  FPI  \
class   category                                                               
object  bottle                              94.8                   91.2 66.0   
        cable                               88.8                   59.8 51.9   
        capsule                             93.5                   83.5 79.9   
        hazelnut                            92.6                   81.3 71.4   
        metal_nut                           85.6                   54.4 72.2   
        pill                                92.7                   83.1 50.4   
        screw                               94.4                   72.6 69.8   
        toothbrush                          93.1                   88.1 60.3   
        transistor                          84.5                   68.5 55.4   
        zipper                              95.9                   84.9 81.2   
        zz average                          91.6                   76.7 65.8   
texture carpet                              96.2                   50.4 21.6   
        grid                                94.6                   91.5 86.0   
        leather                             97.8                   83.7 84.1   
        tile                                86.0                   54.4 42.0   
        wood                                91.1                   64.0 41.7   
        zz average                          93.2                   68.8 55.1   
total   zz average                          92.1                   74.1 62.3   

                    FPI-Poisson            Ours (binary)  \
class   category                                           
object  bottle             79.0   93.0 {\tiny $\pm$ 0.9}   
        cable              55.7   87.6 {\tiny $\pm$ 3.4}   
        capsule            67.6   91.8 {\tiny $\pm$ 0.8}   
        hazelnut           90.9   93.6 {\tiny $\pm$ 0.4}   
        metal_nut          91.5   94.9 {\tiny $\pm$ 0.2}   
        pill               65.2   93.7 {\tiny $\pm$ 0.9}   
        screw              78.4   90.6 {\tiny $\pm$ 1.3}   
        toothbrush         66.8   91.2 {\tiny $\pm$ 0.6}   
        transistor         57.4   72.6 {\tiny $\pm$ 4.4}   
        zipper             86.6   88.9 {\tiny $\pm$ 0.5}   
        zz average         73.9   89.8 {\tiny $\pm$ 0.8}   
texture carpet             93.5  84.0 {\tiny $\pm$ 11.8}   
        grid               95.9   96.5 {\tiny $\pm$ 0.1}   
        leather            98.1   98.9 {\tiny $\pm$ 0.1}   
        tile               83.2   93.9 {\tiny $\pm$ 0.9}   
        wood               81.7   89.2 {\tiny $\pm$ 2.4}   
        zz average         90.5   92.5 {\tiny $\pm$ 2.0}   
total   zz average         79.4   90.7 {\tiny $\pm$ 0.4}   

                           Ours (continous) Ours (logistic continuous)  
class   category                                                        
object  bottle       89.9 {\tiny $\pm$ 1.1}     92.9 {\tiny $\pm$ 0.3}  
        cable        85.4 {\tiny $\pm$ 2.1}     89.9 {\tiny $\pm$ 1.0}  
        capsule      79.9 {\tiny $\pm$ 9.0}     91.4 {\tiny $\pm$ 2.2}  
        hazelnut     93.1 {\tiny $\pm$ 1.3}     93.6 {\tiny $\pm$ 0.9}  
        metal_nut    90.8 {\tiny $\pm$ 1.1}     94.6 {\tiny $\pm$ 0.6}  
        pill         92.5 {\tiny $\pm$ 3.5}     96.0 {\tiny $\pm$ 0.5}  
        screw       80.6 {\tiny $\pm$ 10.3}     90.1 {\tiny $\pm$ 0.3}  
        toothbrush   89.0 {\tiny $\pm$ 1.8}     90.7 {\tiny $\pm$ 1.0}  
        transistor   63.3 {\tiny $\pm$ 1.2}     75.3 {\tiny $\pm$ 2.4}  
        zipper       83.6 {\tiny $\pm$ 3.3}     89.2 {\tiny $\pm$ 0.3}  
        zz average   84.8 {\tiny $\pm$ 2.8}     90.4 {\tiny $\pm$ 0.5}  
texture carpet       71.1 {\tiny $\pm$ 8.2}     85.0 {\tiny $\pm$ 6.2}  
        grid         94.2 {\tiny $\pm$ 0.8}     96.8 {\tiny $\pm$ 0.4}  
        leather      98.6 {\tiny $\pm$ 0.4}     98.7 {\tiny $\pm$ 0.1}  
        tile         90.3 {\tiny $\pm$ 2.5}     95